In [1]:
import os

import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import sgpp, sgml, sgutil

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
expr_dic = {}
for i in ['sunshine', 'cloud', 'dewpoint', 'chp', 'cos_wd', 'sin_wd']:
    for j in range(1, 7):
        expr_dic['{}_{}'.format(i, j)] =  pl.col(i).shift(j).fill_null(strategy = 'backward')
for i in ['pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint', 'humidity', 'cloud', 'sunshine', 'windspeed', 'cos_wd', 'sin_wd', 'chp']:
    expr_dic['{}_ma24'.format(i)] = pl.mean(i).rolling(index_column = 'id', period = '24i', closed = 'left').fill_null(strategy = 'backward')

In [96]:
p1 = make_pipeline(
    sgpp.PolarsProcessor(predefined_types = {'id': pl.Int64}),
    sgpp.ExprProcessor({
        'winddirection': pl.col('winddirection').fill_null(strategy = 'forward'),
        'windspeed': pl.col('windspeed').fill_null(strategy = 'forward')
    }),
    sgpp.ExprProcessor({
        'cos_wd': (pl.col('winddirection') / 180 * np.pi).cos() * pl.col('windspeed'),
        'sin_wd': (pl.col('winddirection') / 180 * np.pi).sin() * pl.col('windspeed'),
        'chp': pl.col('cloud') * pl.col('humidity') / pl.col('pressure'),
        'expected_day': (pl.col('id') % 365) + 1,
        'sin_ed': ((pl.col('id') % 365) / 365 * np.pi).sin(),
        'year': pl.col('id') // 365, 
    })
)
df_train = p1.fit_transform(['data/train.csv'])
df_test = p1.transform(['data/test.csv'])
p2 = make_pipeline(
    sgpp.ExprProcessor(expr_dic),
    sgpp.PandasCoverter(index_col = 'id')
)
df_all = p2.fit_transform(
    pl.concat([df_train, df_test], how = 'align')
)

p3 = make_pipeline(
    sgpp.ApplyWrapper(
        StandardScaler().set_output(transform='pandas'), 
        ['pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint', 'humidity', 'cloud', 'sunshine', 'windspeed'] + ['cos_wd', 'sin_wd', 'chp'] + list(expr_dic.keys()),
    ),
    sgpp.ApplyWrapper(
        MinMaxScaler().set_output(transform='pandas'), ['year', 'expected_day', 'winddirection']
    )
)
df_all = p3.fit_transform(df_all)

df_train = df_all.loc[df_all['rainfall'].notna()]
df_test = df_all.loc[df_all['rainfall'].isna()].drop(columns = ['rainfall'])

In [97]:
df_org = make_pipeline(
    sgpp.PolarsProcessor(predefined_types = {'id': pl.Int64, 'day': pl.Int16, 'rainfall': pl.String}),
).fit_transform(['data/Rainfall.csv']).rename(
    lambda x: x.strip()
)
df_org = make_pipeline(
    sgpp.ExprProcessor({
        'winddirection': pl.col('winddirection').fill_null(strategy = 'forward'),
        'windspeed': pl.col('windspeed').fill_null(strategy = 'forward')
    }),
    sgpp.ExprProcessor({
        'id': pl.arange(1, pl.col('day').len() + 1),
        'sin_ed': (pl.arange(1, pl.col('day').len() + 1) / 365 * np.pi).sin(),
        'cos_wd': (pl.col('winddirection') / 180 * np.pi).cos() * pl.col('windspeed'),
        'sin_wd': (pl.col('winddirection') / 180 * np.pi).sin() * pl.col('windspeed'),
        'chp': pl.col('cloud') * pl.col('humidity') / pl.col('pressure'),
        'rainfall': pl.col('rainfall').replace({'yes': 1, 'no': 0}).cast(pl.Int8),
    }),
    sgpp.ExprProcessor(expr_dic),
    sgpp.PandasCoverter(), 
).fit_transform(df_org).assign(
    expected_day = lambda x: x.index + 1,
    year = -1
).pipe(
    lambda x: x.set_index(-(len(x) - x.index))
)
df_org = p3.transform(df_org)

In [98]:
target = 'rainfall'
sc = sgutil.SGCache('img', 'result')
_ = sc.cache_result(
    'target', lambda : df_train[target].sort_index()
)

In [99]:
len(df_test.columns)

65

In [13]:
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

def get_validation_splitter(validation_fraction):
    return lambda x: train_test_split(x, test_size = validation_fraction)

def include_org(df, include_org = False):
    return pd.concat([df, df_org]) if include_org else df

config = {
    'predict_func': lambda m, df, X: pd.Series(m.predict_proba(df[X])[:, 1], index = df.index),
    'score_func': lambda df, prds: roc_auc_score(df[target], prds),
    'validation_splitter': get_validation_splitter,
    'progress_callback': sgml.ProgressCallBack(), 
    'return_train_scores': True,
    'train_data_proc': include_org,
    'y': target,
}

lr_adapter = sgml.SklearnAdapter(LogisticRegression)
svc_adapter = sgml.SklearnAdapter(SVC)
knn_adapter = sgml.SklearnAdapter(KNeighborsClassifier)
lgb_adapter = sgml.LGBMAdapter(lgb.LGBMClassifier)
xgb_adapter = sgml.XGBAdapter(xgb.XGBClassifier)
cb_adapter = sgml.CBAdapter(cb.CatBoostClassifier)

skf = StratifiedKFold(5, random_state = 123, shuffle = True)
ss = StratifiedShuffleSplit(1, random_state = 123)

In [100]:
from sklearn.feature_selection import SequentialFeatureSelector
X_all = [i for i in df_test.columns.tolist() if i not in ['day']]

# Forward Feature Selection wrapping Logistic Regression

In [9]:
sfs = sc.cache_result(
    'ff_sfs_lr',
    lambda : SequentialFeatureSelector(
        estimator = LogisticRegression(), direction = 'forward', scoring = 'roc_auc', cv = skf, n_jobs = -1
    ).fit(df_train[X_all], df_train[target])
)
np.array(X_all)[sfs.get_support()]

array(['sin_ed', 'pressure', 'maxtemp', 'temparature', 'mintemp',
       'dewpoint', 'sunshine', 'windspeed', 'chp', 'sunshine_1',
       'sunshine_2', 'sunshine_3', 'sunshine_4', 'cloud_3', 'cloud_4',
       'cloud_5', 'cloud_6', 'dewpoint_1', 'dewpoint_5', 'chp_1', 'chp_3',
       'chp_5', 'cos_wd_3', 'cos_wd_4', 'cos_wd_5', 'sin_wd_2',
       'maxtemp_ma24', 'temparature_ma24', 'dewpoint_ma24',
       'humidity_ma24', 'sunshine_ma24', 'year'], dtype='<U16')

In [10]:
hparams = {
    'model_params': {},
    'X_num': np.array(X_all)[sfs.get_support()].tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8972053872053873, 0.012120648342876662)

In [12]:
df_coef = pd.concat([i['coef'] for i in result['model_result']], axis=1).agg(['mean', 'std'], axis = 1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_coef.iloc[:10]

,mean,std,CV
dewpoint_ma24,-0.020234,0.113573,5.612868
maxtemp_ma24,-0.024915,0.081302,3.263118
cos_wd_3,-0.014448,0.043408,3.004357
sunshine_3,-0.018969,0.033022,1.740821
sin_wd_2,-0.015010,0.022889,1.524927
dewpoint_1,0.052131,0.071972,1.380610
chp_1,-0.039389,0.049762,1.263360
maxtemp,0.127891,0.153483,1.200112
temparature_ma24,0.365757,0.205129,0.560833
dewpoint_5,-0.063698,0.031295,0.491304


In [14]:
sc.cv_result('lr_sfs', hparams, lr_adapter, result['valid_prd'])

In [15]:
hparams = {
    'model_params': {},
    'X_num': df_coef.iloc[8:].index.tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8983557800224468, 0.011976255278421333)

In [16]:
df_coef = pd.concat([i['coef'] for i in result['model_result']], axis=1).agg(['mean', 'std'], axis = 1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_coef.iloc[:10]

,mean,std,CV
dewpoint_5,-0.060115,0.045998,0.765164
temparature,-0.148630,0.109471,0.736534
temparature_ma24,0.339612,0.173672,0.511386
cos_wd_5,0.098969,0.045272,0.457440
chp_3,-0.258010,0.096879,0.375485
cloud_6,0.065133,0.023313,0.357921
mintemp,-0.333428,0.118167,0.354401
cloud_5,0.185984,0.061321,0.329710
sunshine_2,0.115192,0.037786,0.328024
cloud_4,0.149698,0.045243,0.302226


In [23]:
sc.cv_result('lr_sfs2', hparams, lr_adapter, result['valid_prd'])

In [19]:
hparams = {
    'model_params': {},
    'X_num': df_coef.iloc[4:].index.tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8985185185185184, 0.01312478134285176)

In [24]:
sc.cv_result('lr_sfs3', hparams, lr_adapter, result['valid_prd'])

In [22]:
df_coef = pd.concat([i['coef'] for i in result['model_result']], axis=1).agg(['mean', 'std'], axis = 1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_coef.iloc[:10]

,mean,std,CV
cloud_6,0.069157,0.025011,0.361659
sunshine_2,0.108589,0.038656,0.355989
chp_3,-0.283220,0.088740,0.313325
cloud_4,0.156865,0.045290,0.288716
pressure,-0.134968,0.037416,0.277223
sin_ed,-0.948355,0.226483,0.238817
sunshine_4,0.216750,0.051374,0.237018
cloud_5,0.232140,0.053793,0.231728
year,0.554469,0.127472,0.229900
chp_5,-0.200099,0.045552,0.227648


# Forward Feature Selection wrapping Logistic Regression Aumenting Org

In [76]:
sfs = sc.cache_result(
    'ff_sfs_lr2',
    lambda : pd.concat([df_train, df_org]).pipe(
        lambda x: SequentialFeatureSelector(
            estimator = LogisticRegression(), direction = 'forward', scoring = 'roc_auc', cv = skf, n_jobs = -1
        ).fit(x[X_all], x[target])
    )
)
np.array(X_all)[sfs.get_support()]

array(['sin_ed', 'pressure', 'maxtemp', 'temparature', 'mintemp',
       'dewpoint', 'humidity', 'cloud', 'sunshine', 'windspeed', 'cos_wd',
       'chp', 'sunshine_1', 'sunshine_2', 'sunshine_3', 'sunshine_4',
       'sunshine_5', 'cloud_2', 'cloud_3', 'cloud_4', 'dewpoint_2',
       'dewpoint_6', 'chp_2', 'chp_4', 'chp_6', 'cos_wd_3', 'sin_wd_2',
       'maxtemp_ma24', 'temparature_ma24', 'mintemp_ma24', 'year',
       'winddirection'], dtype='<U16')

In [77]:
hparams = {
    'model_params': {},
    'X_num': np.array(X_all)[sfs.get_support()].tolist(),
    'train_data_proc_param': {'include_org': True}
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
sc.cv_result('lr_sfs_a', hparams, lr_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8947811447811448, 0.012302086080667738)

In [78]:
df_coef = pd.concat([i['coef'] for i in result['model_result']], axis=1).agg(['mean', 'std'], axis = 1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_coef.iloc[:10]

,mean,std,CV
maxtemp,-0.021097,0.149820,7.101543
cos_wd_3,-0.010459,0.045744,4.373780
mintemp_ma24,0.040715,0.160246,3.935820
sunshine_5,0.019164,0.033826,1.765062
chp_6,-0.022549,0.038773,1.719536
sin_wd_2,-0.022583,0.030652,1.357292
maxtemp_ma24,0.119042,0.137336,1.153678
chp_4,0.042262,0.034726,0.821676
chp,0.069188,0.052726,0.762075
temparature,-0.166954,0.126749,0.759183


In [79]:
hparams = {
    'model_params': {},
    'X_num': df_coef.iloc[6:].index.tolist(),
    'train_data_proc_param': {'include_org': True}
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
sc.cv_result('lr_sfs_a2', hparams, lr_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8962906846240178, 0.01146601290644487)

In [80]:
df_coef = pd.concat([i['coef'] for i in result['model_result']], axis=1).agg(['mean', 'std'], axis = 1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_coef.iloc[:10]

,mean,std,CV
chp_4,0.034988,0.042251,1.207573
chp,0.054333,0.046949,0.864086
sunshine_3,0.048751,0.035849,0.735355
dewpoint_2,-0.100342,0.063914,0.636965
temparature,-0.184064,0.098829,0.536927
maxtemp_ma24,0.118299,0.062029,0.524339
chp_2,-0.134452,0.053239,0.395968
year,0.290991,0.111759,0.384062
cos_wd,-0.257909,0.063183,0.244983
dewpoint_6,0.340524,0.075403,0.221431


In [82]:
hparams = {
    'model_params': {},
    'X_num': df_coef.iloc[7:].index.tolist(),
    'train_data_proc_param': {'include_org': True}
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
sc.cv_result('lr_sfs_a3', hparams, lr_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8971043771043771, 0.011150089212136121)

# Backward Feature Selection wrapping Logistic Regression

In [107]:
sfs = sc.cache_result(
    'bf_sfs_lr',
    lambda : SequentialFeatureSelector(
        estimator = LogisticRegression(), direction = 'backward', scoring = 'roc_auc', cv = skf, n_jobs = -1
    ).fit(df_train[X_all], df_train[target]), rerun = False
)
np.array(X_all)[sfs.get_support()]

array(['sin_ed', 'pressure', 'maxtemp', 'temparature', 'mintemp',
       'dewpoint', 'sunshine', 'windspeed', 'cos_wd', 'sin_wd', 'chp',
       'sunshine_1', 'sunshine_2', 'sunshine_4', 'cloud_3', 'cloud_4',
       'cloud_5', 'dewpoint_5', 'chp_3', 'chp_4', 'chp_5', 'cos_wd_3',
       'cos_wd_4', 'sin_wd_3', 'temparature_ma24', 'mintemp_ma24',
       'cloud_ma24', 'sunshine_ma24', 'cos_wd_ma24', 'sin_wd_ma24',
       'chp_ma24', 'year'], dtype='<U16')

In [108]:
hparams = {
    'model_params': {},
    'X_num': np.array(X_all)[sfs.get_support()].tolist(),
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
sc.cv_result('lr_bfs', hparams, lr_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8971717171717172, 0.011681897853436847)

# Backward Feature Selection wrapping Logistic Regression Augumenting Org

In [109]:
sfs = sc.cache_result(
    'bf_sfs_lr2',
    lambda : pd.concat([df_train, df_org]).pipe(
        lambda x: SequentialFeatureSelector(
            estimator = LogisticRegression(max_iter = 500), direction = 'backward', scoring = 'roc_auc', cv = skf, n_jobs = -1
        ).fit(x[X_all], x[target])
    )
)
np.array(X_all)[sfs.get_support()]

array(['sin_ed', 'pressure', 'mintemp', 'dewpoint', 'humidity', 'cloud',
       'sunshine', 'windspeed', 'cos_wd', 'sin_wd', 'chp', 'sunshine_1',
       'sunshine_3', 'sunshine_4', 'sunshine_6', 'cloud_4', 'cloud_5',
       'dewpoint_2', 'dewpoint_6', 'chp_3', 'chp_5', 'chp_6', 'cos_wd_3',
       'sin_wd_1', 'sin_wd_3', 'sin_wd_5', 'temparature_ma24',
       'mintemp_ma24', 'cos_wd_ma24', 'sin_wd_ma24', 'year',
       'winddirection'], dtype='<U16')

In [110]:
hparams = {
    'model_params': {},
    'X_num': np.array(X_all)[sfs.get_support()].tolist(),
    'train_data_proc_param': {'include_org': True}
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
sc.cv_result('lr_bfs_a', hparams, lr_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8929517396184063, 0.009206929636066645)

# Forward Feature Selection wrapping LGBM

In [15]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': X_all
}
result = sgml.cv(df_train, skf, hparams, config, lgb_adapter)
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8884736251402916, 0.9403738776655443)

In [18]:
sfs = sc.cache_result(
    'ff_sfs_lgb',
    lambda : SequentialFeatureSelector(
        estimator = lgb.LGBMClassifier(verbose = 0, **hparams['model_params']), direction = 'forward', scoring = 'roc_auc', cv = skf
    ).fit(df_train[X_all], df_train[target])
)
np.array(X_all)[sfs.get_support()]

array(['sin_ed', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',
       'sunshine', 'windspeed', 'sin_wd', 'chp', 'cloud_1', 'cloud_2',
       'cloud_3', 'cloud_5', 'cloud_6', 'dewpoint_1', 'dewpoint_2',
       'dewpoint_4', 'chp_3', 'chp_5', 'cos_wd_3', 'cos_wd_4', 'cos_wd_5',
       'sin_wd_2', 'sin_wd_6', 'maxtemp_ma24', 'temparature_ma24',
       'mintemp_ma24', 'dewpoint_ma24', 'humidity_ma24', 'sunshine_ma24',
       'year', 'expected_day'], dtype='<U16')

In [21]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': np.array(X_all)[sfs.get_support()].tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lgb_adapter)
sc.cv_result('lgb_sfs', hparams, lgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8969191919191918, 0.9363073442760943)

In [33]:
df_imp = pd.concat(
    [i['feature_importance'] for i in result['model_result']], axis=1
).agg(['mean', 'std'], axis=1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_imp.iloc[:10]

,mean,std,CV
sin_wd_6,8.2,7.049823,0.859734
cloud_5,6.2,4.764452,0.768460
dewpoint_ma24,13.0,9.772410,0.751724
cos_wd_5,8.2,6.058052,0.738787
cos_wd_3,9.4,6.655825,0.708066
mintemp,7.8,5.069517,0.649938
dewpoint_4,12.6,7.893035,0.626431
chp_5,5.4,3.361547,0.622509
expected_day,9.4,5.319774,0.565933
sunshine_ma24,24.6,12.218838,0.496701


In [40]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': df_imp.iloc[3:].index.tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lgb_adapter)
sc.cv_result('lgb_sfs2', hparams, lgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8983277216610549, 0.9354950547138048)

In [41]:
df_imp = pd.concat(
    [i['feature_importance'] for i in result['model_result']], axis=1
).agg(['mean', 'std'], axis=1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_imp.iloc[:10]

,mean,std,CV
year,3.6,2.701851,0.750514
cos_wd_5,10.0,7.211103,0.721110
dewpoint_1,6.4,4.335897,0.677484
cos_wd_3,10.4,6.877500,0.661298
mintemp,9.0,5.567764,0.618640
mintemp_ma24,10.4,6.148170,0.591170
cloud_2,4.0,2.121320,0.530330
sin_wd_2,6.4,3.361547,0.525242
sunshine_ma24,23.6,11.760102,0.498309
temparature,4.4,2.190890,0.497930


# Forward Feature Selection wrapping LGBM Augumenting Org

In [115]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': X_all
}
sfs = sc.cache_result(
    'ff_sfs_lgb2',
    lambda : pd.concat([df_train, df_org]).pipe(
        lambda x: SequentialFeatureSelector(
            estimator = lgb.LGBMClassifier(verbose = 0, **hparams['model_params']), direction = 'forward', scoring = 'roc_auc', cv = skf
        ).fit(x[X_all], x[target])
    )
)
np.array(X_all)[sfs.get_support()]

array(['sin_ed', 'pressure', 'maxtemp', 'temparature', 'mintemp',
       'dewpoint', 'humidity', 'cloud', 'sunshine', 'windspeed', 'chp',
       'sunshine_1', 'sunshine_5', 'cloud_2', 'cloud_3', 'cloud_6',
       'dewpoint_1', 'dewpoint_2', 'chp_2', 'chp_3', 'chp_4', 'chp_6',
       'cos_wd_1', 'sin_wd_5', 'maxtemp_ma24', 'temparature_ma24',
       'dewpoint_ma24', 'humidity_ma24', 'cloud_ma24', 'windspeed_ma24',
       'chp_ma24', 'year'], dtype='<U16')

In [116]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': np.array(X_all)[sfs.get_support()].tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lgb_adapter)
sc.cv_result('lgb_sfs_a', hparams, lgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8902076318742985, 0.9356239478114479)

In [117]:
df_imp = pd.concat(
    [i['feature_importance'] for i in result['model_result']], axis=1
).agg(['mean', 'std'], axis=1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_imp.iloc[:10]

,mean,std,CV
temparature,3.0,3.937004,1.312335
sin_ed,3.2,4.147288,1.296028
year,4.4,4.979960,1.131809
cloud_2,1.8,1.643168,0.912871
sunshine_5,8.6,6.693280,0.778288
sin_wd_5,8.6,6.580274,0.765148
chp_2,4.4,3.209361,0.729400
humidity,12.2,8.318654,0.681857
cloud_6,5.2,3.492850,0.671702
chp_4,5.4,3.286335,0.608581


In [128]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': df_imp.iloc[6:].index.tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lgb_adapter)
sc.cv_result('lgb_sfs_a2', hparams, lgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8937485970819304, 0.935530303030303)

# Backward Feature Selection wrapping LGBM

In [111]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': X_all
}
sfs = sc.cache_result(
    'bf_sfs_lgb',
    lambda : SequentialFeatureSelector(
        estimator = lgb.LGBMClassifier(verbose = 0, **hparams['model_params']), direction = 'backward', scoring = 'roc_auc', cv = skf
    ).fit(df_train[X_all], df_train[target])
)
np.array(X_all)[sfs.get_support()]

array(['temparature', 'mintemp', 'sunshine', 'windspeed', 'cos_wd', 'chp',
       'sunshine_1', 'sunshine_6', 'cloud_1', 'cloud_5', 'cloud_6',
       'dewpoint_3', 'chp_1', 'chp_2', 'chp_3', 'chp_5', 'cos_wd_3',
       'cos_wd_4', 'sin_wd_3', 'sin_wd_4', 'sin_wd_6', 'pressure_ma24',
       'maxtemp_ma24', 'temparature_ma24', 'dewpoint_ma24',
       'humidity_ma24', 'cloud_ma24', 'sunshine_ma24', 'sin_wd_ma24',
       'chp_ma24', 'year', 'winddirection'], dtype='<U16')

In [113]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': np.array(X_all)[sfs.get_support()].tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lgb_adapter)
sc.cv_result('lgb_bfs', hparams, lgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8956593714927049, 0.9357386363636364)